In [1]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [2]:
from collections import namedtuple

In [3]:
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])
gamma = 0.99

In [4]:
class Network222(nn.Module):
  def __init__(self):
    super(Network222, self).__init__()
    self.clayer = nn.Linear(4, 128)

    self.ahead = nn.Linear(128, 2)

    self.chead = nn.Linear(128, 1)

    self.saved_actions = []
    self.rewards = []

  def forward(self, state):
    x = F.relu(self.clayer(state))
   # print('== forward == ', x)
    probs = F.softmax(self.ahead(x), dim=-1)
    #print('==probs== ', probs)
    value = self.chead(x)
    return probs, value

In [13]:
env = gym.make('CartPole-v0')

In [14]:
def select_action(state):
 # print('debuging----')
#  print(state)
  state = torch.from_numpy(state).float()
  #print(state)
  probs, state_value = model(state)
  #print(probs)
  m = Categorical(probs)
  #print(m)
  action = m.sample()
  model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
  return action.item()


In [15]:
def select_action2(state):
    state = torch.from_numpy(state).float()
    #to_device(state, device)
    probs, state_value = model(state)

    # create a categorical distribution over the list of probabilities of actions
    m = Categorical(probs)

    # and sample an action using the distribution
    action = m.sample()

    # save to action buffer
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))

    # the action to take (left or right)
    return action.item()

In [16]:
class Policy(nn.Module):
    """
    implements both actor and critic in one model
    """
    def __init__(self):
      
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)

         # actor's layer
        self.layer2 = nn.Linear(128, 128)
        # actor's layer
        self.action_head = nn.Linear(128, 2)

        # critic's layer
        self.value_head = nn.Linear(128, 1)

        # action & reward buffer
        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        """
        forward of both actor and critic
        """
       # to_device(x, device)
        print(x)
        x = F.relu(self.affine1(x))
        x = F.relu(self.layer2(x))

        # actor: choses action to take from state s_t 
        # by returning probability of each action
        action_prob = F.softmax(self.action_head(x), dim=-1)

        # critic: evaluates being in the state s_t
        state_values = self.value_head(x)

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t 
        return action_prob, state_values

In [17]:
def getReturns():
    R = 0
    returns = [] # list to save the true values

    # calculate the true value using rewards returned from the environment
    # 
    for r in model.rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)

    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    return returns

In [18]:
model = Policy()
eps = np.finfo(np.float32).eps.item()
optimizer = optim.Adam(model.parameters(), lr=3e-2)

In [19]:
def learn():
  R = 0
  saved_actions = model.saved_actions
  policy_losses = []
  value_losses = []
  returns = getReturns()
    
  # Normalize
  returns = torch.tensor(returns)
  returns = (returns - returns.mean()) / (returns.std() + eps)

  for (prob, value), r in zip(saved_actions, returns):
    advance = r - value.item()

    policy_losses.append(-prob*advance)
    value_losses.append(F.smooth_l1_loss(value, torch.tensor([r])))
  optimizer.zero_grad()
  loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
  loss.backward()
  optimizer.step()

  del model.rewards[:]
  del model.saved_actions[:]   



In [20]:
def finish_episode():
    """
    Training code. Calculates actor and critic loss and performs backprop.
    """
    R = 0
    saved_actions = model.saved_actions
    policy_losses = [] # list to save actor (policy) loss
    value_losses = [] # list to save critic (value) loss
    returns = getReturns()
   # print('1...')
   # print(returns)
    i1 = 0
    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()
       # if i1 == 0: 
          #  print('2...')
         #   print(log_prob, value, R)
       # i1 = i1 + 1
        # calculate actor (policy) loss 
        policy_losses.append(-log_prob * advantage)

        # calculate critic (value) loss using L1 smooth loss
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))

    # reset gradients
    optimizer.zero_grad()

    # sum up all the values of policy_losses and value_losses
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

    # perform backprop
    loss.backward()
    optimizer.step()

    # reset rewards and action buffer
    del model.rewards[:]
    del model.saved_actions[:]

In [21]:
running_reward = 10
for i in range(10):

  state = env.reset()
  ep_reward = 0
  for t in range(1000):
    action = select_action2(state)

    new_state, reward, done, _ = env.step(action)

    model.rewards.append(reward)
    running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    finish_episode()

    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                i, ep_reward, running_reward))

    state = new_state

tensor([-0.0235, -0.0475, -0.0303, -0.0095])
Episode 0	Last reward: 0.00	Average reward: 9.50
tensor([-0.0245, -0.2422, -0.0305,  0.2734])


RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

In [ ]:
env.reset()

In [225]:
def main():
    running_reward = 10

    # run inifinitely many episodes
    for i_episode in range(10):

        # reset environment and episode reward
        state = env.reset()
        ep_reward = 0

        # for each episode, only run 9999 steps so that we don't 
        # infinite loop while learning
        for t in range(1, 10000):

            # select action from policy
            action = select_action(state)

            # take the action
            state, reward, done, _ = env.step(action)

            #if args.render:
            #env.render()

            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break

        # update cumulative reward
        running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward

        # perform backprop
        finish_episode()

        # log results
        if i_episode % log_interval == 0:
            print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(
                  i_episode, ep_reward, running_reward))

        # check if we have "solved" the cart pole problem
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

In [226]:
if __name__ == '__main__':
    main()

tensor([-0.0337, -0.0285, -0.0118, -0.0166])


RuntimeError: ignored